In [485]:
def get_word_count(filename):
  with open(filename, 'r') as f:
    # Read the entire file content
    content = f.read()

    # Split the content into words, removing whitespaces
    words = content.split()

    return len(words)

# Example usage
filename = "./raw_data/combined_unlabeled_commentary.txt"  # Replace with your actual filename
word_count = get_word_count(filename)
print(f"The file '{filename}' has {word_count} words.")
# print(f"The file '{filename2}' has {word_count2} words.")

The file './raw_data/combined_unlabeled_clean.txt' has 1855918 words.


In [314]:
def split_sentences(input_file, output_file):
  """
  Splits a text file into separate lines for each sentence.

  Args:
    input_file (str): Path to the input text file.
    output_file (str): Path to the output file where the split sentences will be written.
  """
  sentences = []
  current_sentence = ""
  with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
      # Add line to current sentence (excluding trailing newline)
      current_sentence += line.strip()

      # Check for sentence delimiters (e.g., ., ?, !) and write complete sentences
      for delimiter in [".", "?", "!"]:
        if delimiter in current_sentence:
          if (len(current_sentence) > 8):
            sentences.append(current_sentence.split(delimiter)[0] + delimiter + "\n")
          current_sentence = current_sentence.split(delimiter)[1].strip()

      # Write any remaining sentence at the end of the file
      # if current_sentence:
      #   sentences.append(current_sentence)
    outfile.writelines(sentences)

# Example usage:
input_file = "./raw_data/combined_unlabeled_commentary.txt"
output_file = "./raw_data/combined_commentary_split.txt"  # Output file with split sentences

split_sentences(input_file, output_file)

print(f"Text file split into sentences and saved to {output_file}")

Text file split into sentences and saved to ./raw_data/combined_commentary_split.txt


In [315]:
word_count = get_word_count(output_file)
word_count

# dataset

1245988

In [480]:
from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling
from datasets import load_dataset
from torch.utils.data import DataLoader

import torch

# Define paths and hyperparameters
model_name = "bert-base-uncased"  # Pre-trained BERT model
text_file = "./raw_data/combined_commentary_split.txt"  # Path to your text file
batch_size = 8  # Adjust this based on your GPU memory
max_len = 512  # Maximum sequence length
learning_rate = 1e-5

# Check for GPU availability
device = torch.device("mps")

# Load tokenizer and model on chosen device
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [481]:
# Define function to mask tokens
def mask_sentence(sentence):
  """Masks 15% of the tokens in a sentence."""
  #print(sentence["text"])
  masked_sentence = tokenizer.batch_encode_plus(sentence["text"], return_tensors="pt", truncation = True, padding="max_length", max_length=280)
  masked_sentence = masked_sentence.to(device)  # Move data to device

  input_ids = masked_sentence["input_ids"]
  attention_mask = masked_sentence["attention_mask"]
  labels = input_ids.clone()

  # Randomly mask 15% of tokens
  probability_mask = 0.15
  mask_indices = torch.bernoulli(torch.ones_like(input_ids) * probability_mask).bool()
  labels[~mask_indices] = -100  # Set padding tokens to -100

  # Replace masked tokens with [MASK]
  input_ids[mask_indices] = tokenizer.mask_token_id

  print("check")
  print(input_ids.shape)
  print(attention_mask.shape)

  return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}
    
# def collate_fn(batch):

In [482]:
# Load data from text file
small_file = "./raw_data/combined_unlabeled_commentary_clean.txt"
large_file = "./raw_data/combined_commentary_split.txt"
dataset = load_dataset("text", data_files={"train": large_file}, split="train")
train_data = dataset.map(mask_sentence, batched=True)
train_data.set_format("torch")
print(dataset)
# for i in dataset:
#     print(i)
# print(type(train_data))
# print(train_data)
# print (len(train_data["text"]))
# print (len(train_data["input_ids"]))
# print (len(train_data["attention_mask"]))
# print (len(train_data["labels"]))

# print (type(train_data["text"]))
# print (type(train_data["input_ids"]))
# print (type(train_data["attention_mask"]))
# print (type(train_data["labels"]))

# print (train_data["attention_mask"].shape)
# print (train_data["attention_mask"][0].shape)

# print (train_data["attention_mask"][50].shape)
# print (train_data["attention_mask"][99].shape)

train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
num_batches = len(train_loader)
print(f"Number of batches: {num_batches}")

print(dataset)
print(dataset[0])

print(train_loader)

Dataset({
    features: ['text'],
    num_rows: 33568
})
Number of batches: 8392
Dataset({
    features: ['text'],
    num_rows: 33568
})
{'text': 'INSIGHTSPre Match1Blackpool have won just two of their last 15 league games against Sheffield United (D6 L7), although they are unbeaten in four against the Blades (W1 D3).'}


In [460]:
# num = 0
# for i in train_dataloader:
#     print(i["input_ids"].shape)
#     #if num%100:
#         #print(i["input_ids"].shape)
#     # num+=1

In [483]:
for epoch in range(3):  # Adjust number of epochs
  i=0
  for batch in train_loader:
    #print(batch)
    # Move data to device
    #batch.to(device)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    #print(batch["text"])

    # Forward pass
    outputs = model(input_ids= input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress (optional)
    if i%100 == 0:
      print(f"Epoch: {epoch+1}, Loss: {loss}")
    i+=1;

# Save the fine-tuned model (optional)
model.save_pretrained("masked_lm_model")

Epoch: 1, Loss: 14.641313552856445
Epoch: 1, Loss: 1.7769700288772583
Epoch: 1, Loss: 1.135185718536377
Epoch: 1, Loss: 0.697974443435669
Epoch: 1, Loss: 0.573076069355011
Epoch: 1, Loss: 0.45633938908576965
Epoch: 1, Loss: 0.5330857634544373
Epoch: 1, Loss: 0.20004169642925262
Epoch: 1, Loss: 0.655945897102356
Epoch: 1, Loss: 0.35381612181663513
Epoch: 1, Loss: 0.6889758706092834
Epoch: 1, Loss: 0.4797069728374481
Epoch: 1, Loss: 0.3986416161060333
Epoch: 1, Loss: 0.6535927653312683
Epoch: 1, Loss: 0.33146506547927856
Epoch: 1, Loss: 0.24999576807022095
Epoch: 1, Loss: 0.21215470135211945
Epoch: 1, Loss: 0.42111289501190186
Epoch: 1, Loss: 0.27609488368034363
Epoch: 1, Loss: 0.34636086225509644
Epoch: 1, Loss: 0.3873303234577179
Epoch: 1, Loss: 0.7174456715583801
Epoch: 1, Loss: 0.40976354479789734
Epoch: 1, Loss: 0.3213740885257721
Epoch: 1, Loss: 0.5686247944831848
Epoch: 1, Loss: 0.3610919415950775
Epoch: 1, Loss: 0.6671115756034851
Epoch: 1, Loss: 0.43717530369758606
Epoch: 1, Los

In [474]:
%reset -f
